# UDLF Non-Local Means (RL-RECOM)
Images and data collected from experiments of UDLF Non-Local Means with RL-RECOM as the reranking method used.

## Imports

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

from skimage import data
from skimage.color import rgb2gray
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.restoration import estimate_sigma
from skimage.util import random_noise

from nlmeans.nlmeans import nlmeans
from nlmeans.nlmeans_udlf import nlmeans_udlf

## Parameters

In [16]:
# Non-Local Means parameters
hW = 10        # Window Size
hP = 3         # Patch Size
tau = 0.15     # Contribution of similarity on noisy-data
sig = 0.1      # Standard deviation of the gaussian noise
shape = 'disk' # Patch shape
n_w = None     # Number of weights to use or use all(None)

# Image to test with
image = 'astronaut'

# UDLF Parameters
udl_method = 'RLRECOM'
udl_params = {
    'k': 8,
    'lambda': 2,
    'epsilon': 0.0125
}

## Image preprocessing
Get the image from skimage dataset or use the image from the local file system if not found in skimage. 

After that, do the necessary preprocessing:
- Crop the image to be 200x200 (This is necessary because UDLF has to be able to allocate memory to process the 
  ranked lists gotten from the image. In a system with more memory the image can possibly be greater);
- Convert the image from RGB to Grayscale.

In [17]:
images = ('astronaut',
          'brick',
          'camera',
          'cat',
          'checkerboard',
          'clock',
          'coffee',
          'coins',
          'grass',
          'gravel',
          'horse')

if image in images:
    im = getattr(data, image)()
    im = im[100:300, 100:300]
    im_name = image
else:
    im = plt.imread(image).astype('float')
    im_name = os.path.splitext(os.path.basename(image))[0]

if len(im.shape) == 3:
    im = rgb2gray(im)

# Show the image
plt.imshow(im, cmap='gray')

In [18]:
im_nse = random_noise(im, var = sig**2)
sigma_est = np.mean(estimate_sigma(im_nse, channel_axis=-1))
plt.imshow(im_nse, cmap='gray')

## Non-Local Means SAP
For comparison the Non-Local Means SAP (the algorithm on which UDLF Non-Local Means is based) is run as well. 

In [19]:
%%time
im_fil1 = nlmeans(im_nse, hW, hP, tau, sigma_est, shape)
plt.imshow(im_fil1, cmap='gray')

## UDLF Non-Local Means (RL-RECOM)
This denoise the image using RL-RECOM as the reranking method and considering RLRECOM default parameters as indicated in the [UDLF config.ini](https://github.com/UDLF/UDLF/blob/master/bin/config.ini)

In [20]:
%%time
im_fil2 = nlmeans_udlf(im_nse, hW, hP, tau, sigma_est, shape, udl_method, udl_params, n_w)
plt.imshow(im_fil2, cmap='gray')

### Show the images side by side

In [21]:
plt.rcParams['figure.figsize'] = [15, 5]
plt.figure()
plt.subplot(1, 3, 1)
plt.xlabel('Noise image')
plt.imshow(im_nse, cmap='gray')
plt.subplot(1, 3, 2)
plt.xlabel('Nlmeans')
plt.imshow(im_fil1, cmap='gray')
plt.subplot(1, 3, 3)
plt.xlabel('RL-RECOM Nlmeans')
plt.imshow(im_fil2, cmap='gray')
plt.show()

## Evaluation of results
The following values are calculated to each of the three images:
- PSNR
- SSIM
- MSE

In [22]:
im_noise_psnr   = psnr(im, im_nse, data_range=im_nse.max() - im_nse.min())
im_noise_ssim   = ssim(im, im_nse, data_range=im_nse.max() - im_nse.min())
im_noise_mse    = mse(im, im_nse)

nlm_noise_psnr  = psnr(im, im_fil1, data_range=im_fil1.max() - im_fil1.min())
nlm_noise_ssim  = ssim(im, im_fil1, data_range=im_fil1.max() - im_fil1.min())
nlm_noise_mse   = mse(im, im_fil1)

udlf_noise_psnr = psnr(im, im_fil2, data_range=im_fil2.max() - im_fil2.min())
udlf_noise_ssim = ssim(im, im_fil2, data_range=im_fil2.max() - im_fil2.min())
udlf_noise_mse  = mse(im, im_fil2)

pd.DataFrame(
    data   = np.array([[im_noise_psnr, nlm_noise_psnr, udlf_noise_psnr],
                       [im_noise_ssim, nlm_noise_ssim, udlf_noise_ssim],
                       [im_noise_mse,  nlm_noise_mse,  udlf_noise_mse]]),
    index   = ['PSNR', 'SSIM', 'MSE'],
    columns = ['Noise', 'Nlmeans', 'RLRECOM Nlmeans'],
)

## Hyperparameters Tuning